# Finding Detection Mistakes with FiftyOne

Annotations mistakes create an artificial ceiling on the performance of your models. However, finding these mistakes by hand is at least as arduous as the original annotation work! Enter FiftyOne.

In [this tutorial](https://voxel51.com/docs/fiftyone/tutorials/detection_mistakes.html), we explore how FiftyOne can be used to help you find mistakes in your object detection annotations. To detect mistakes in classification datasets, check out [this tutorial](https://voxel51.com/docs/fiftyone/tutorials/classification_mistakes.html).

We'll cover the following concepts:

- Loading your existing dataset [into FiftyOne](https://voxel51.com/docs/fiftyone/user_guide/dataset_creation/index.html)
- [Adding model predictions](https://voxel51.com/docs/fiftyone/recipes/adding_detections.html) to your dataset
- Computing insights into your dataset relating to [possible label mistakes](https://voxel51.com/docs/fiftyone/user_guide/brain.html#label-mistakes)
- Visualizing mistakes in the [FiftyOne App](https://voxel51.com/docs/fiftyone/user_guide/app.html)

**So, what's the takeaway?**

FiftyOne can help you find and correct label mistakes in your datasets, enabling you to curate higher quality datasets and, ultimately, train better models!

## Setup

In order to compute mistakenness, your dataset needs to have two [detections fields](https://voxel51.com/docs/fiftyone/user_guide/using_datasets.html#object-detection), one with your ground truth annotations and one with your model predictions.

In [1]:
import json

import fiftyone as fo
import fiftyone.brain as fob
from fiftyone import ViewField as F

In [2]:
# A name for the dataset
name = "my-dataset"

# The directory containing the source images
data_path = "/media/data/datasets/TIL2021_CV_dataset/images/c3_test/images"

# The path to the COCO labels JSON file
labels_path = "/media/data/datasets/TIL2021_CV_dataset/images/c3_test/c3_test.json"

# The path to the COCO predictions JSON file
preds_path = "/media/data/datasets/TIL2021_CV_dataset/images/c3_test/til21_c3_pred.json"

In [3]:
with open(labels_path) as json_file:
    labels_data = json.load(json_file)
    
with open(preds_path) as json_file:
    preds_data = json.load(json_file)

In [4]:
# run this if getting Dataset already exists error in the next step
# fo.load_dataset(name).delete()

In [5]:
# Import the dataset
dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path=data_path,
    labels_path=labels_path,
    name=name,
)

 100% |█████████████████| 400/400 [955.2ms elapsed, 0s remaining, 418.7 samples/s]      


In [6]:
print(dataset)

Name:        my-dataset
Media type:  image
Num samples: 400
Persistent:  False
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)


## Add predictions to dataset

In [7]:
# Add predictions to samples
cat2name = {cat['id']:cat['name'] for cat in labels_data["categories"]}

with fo.ProgressBar() as pb:
    for sample in pb(dataset):
        filepath = sample.filepath
        filename = filepath.split('/')[-1]

        # get image_id of sample
        img_data = next(item for item in labels_data["images"] if item["file_name"] == filename)
        image_id = img_data["id"]
        img_w = img_data["width"]
        img_h = img_data["height"]

        # Convert detections to FiftyOne format
        detections = []
        for pred in (x for x in preds_data if x["image_id"] == image_id):
            # Convert to [top-left-x, top-left-y, width, height] in relative coordinates in [0, 1] x [0, 1]
            x, y, w, h = pred["bbox"]
            rel_box = [x / img_w, y / img_h, w / img_w, h / img_h]

            detections.append(
                fo.Detection(
                    label=cat2name[pred["category_id"]],
                    bounding_box=rel_box,
                    confidence=pred["score"]
                )
            )

        # Save predictions to dataset
        sample["predictions"] = fo.Detections(detections=detections)
        sample.save()

 100% |█████████████████| 400/400 [4.9s elapsed, 0s remaining, 77.6 samples/s]       


In [8]:
# Print a sample ground truth detection
sample = dataset.first()
print(sample.predictions.detections[0])

<Detection: {
    'id': '60e4137dd10584dfdd7dbd02',
    'attributes': BaseDict({}),
    'tags': BaseList([]),
    'label': 'Chicken',
    'bounding_box': BaseList([
        0.391357421875,
        0.2001953125,
        0.59716796875,
        0.7936197916666666,
    ]),
    'mask': None,
    'confidence': 0.907,
    'index': None,
}>


Let's start by visualizing the dataset in the [FiftyOne App](https://voxel51.com/docs/fiftyone/user_guide/app.html):

In [9]:
# Launch the App in a dedicated browser tab
session = fo.launch_app(dataset, auto=False)
session.open_tab()

Session launched. Run `session.show()` to open the App in a cell output.


When working with FiftyOne datasets that contain a field with `Detections`, you can create a [patches view](https://voxel51.com/docs/fiftyone/user_guide/app.html#viewing-object-patches) both through Python and directly in the FiftyOne App to view each detection as a separate sample.

In [10]:
patches_view = dataset.to_patches("ground_truth")
print(patches_view)

Dataset:     my-dataset
Media type:  image
Num patches: 572
Tags:        []
Patch fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    sample_id:    fiftyone.core.fields.ObjectIdField
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detection)
View stages:
    1. ToPatches(field='ground_truth')


In [11]:
# the app view updates automatically after running this
session.view = patches_view

Alternatively, open the App and click the [patches button](https://voxel51.com/docs/fiftyone/user_guide/app.html#viewing-object-patches), then select `ground_truth` to create the same view that we created above.

## Compute mistakenness

Now we're ready to assess the mistakenness of the ground truth detections.

We can do so by running the [compute_uniqueness()](https://voxel51.com/docs/fiftyone/api/fiftyone.brain.html#fiftyone.brain.compute_mistakenness) method from the FiftyOne Brain:

In [12]:
# Compute mistakenness of annotations in `ground_truth` field using 
# predictions from `predictions` field as point of reference
fob.compute_mistakenness(dataset, "predictions", label_field="ground_truth")

Evaluating detections...
 100% |█████████████████| 400/400 [10.0s elapsed, 0s remaining, 44.9 samples/s]      
Computing mistakenness...
 100% |█████████████████| 400/400 [8.7s elapsed, 0s remaining, 48.9 samples/s]       
Mistakenness computation complete


The above method populates a number of fields on the samples of our dataset as well as the ground truth and predicted objects:

New ground truth object attributes (in `ground_truth` field):

- `mistakenness` (float): A measure of the likelihood that a ground truth object's label is incorrect
- `mistakenness_loc`: A measure of the likelihood that a ground truth object's localization (bounding box) is inaccurate
- `possible_spurious`: Ground truth objects that were not matched with a predicted object and are deemed to be likely spurious annotations will have this attribute set to True

New predicted object attributes (in `predictions` field):

- `possible_missing`: If a highly confident prediction with no matching ground truth object is encountered, this attribute is set to True to indicate that it is a likely missing ground truth annotation

Sample-level fields:

- `mistakenness`: The maximum mistakenness of the ground truth objects in each sample
- `possible_spurious`: The number of possible spurious ground truth objects in each sample
- `possible_missing`: The number of possible missing ground truth objects in each sample

In [13]:
# inspect the first sample in the dataset
print(dataset.first())

<Sample: {
    'id': '60e4137cd10584dfdd7db6fa',
    'media_type': 'image',
    'filepath': '/media/data/datasets/TIL2021_CV_dataset/images/c3_test/images/1.jpg',
    'tags': BaseList([]),
    'metadata': <ImageMetadata: {
        'size_bytes': None,
        'mime_type': None,
        'width': 1024,
        'height': 768,
        'num_channels': None,
    }>,
    'ground_truth': <Detections: {
        'detections': BaseList([
            <Detection: {
                'id': '60e4137cd10584dfdd7db6f4',
                'attributes': BaseDict({}),
                'tags': BaseList([]),
                'label': 'Chicken',
                'bounding_box': BaseList([
                    0.73994140625,
                    0.38212239583333335,
                    0.26005859375,
                    0.48194010416666666,
                ]),
                'mask': None,
                'confidence': None,
                'index': None,
                'supercategory': '',
                'iscrowd': 

## Analyzing the results

Let's use FiftyOne to investigate the results.

First, let's show the samples with the most likely annotation mistakes:

In [14]:
# Sort by likelihood of mistake (most likely first)
mistake_view = dataset.sort_by("mistakenness", reverse=True)

# Print some information about the view
print(mistake_view)

Dataset:     my-dataset
Media type:  image
Num samples: 400
Tags:        []
Sample fields:
    id:                fiftyone.core.fields.ObjectIdField
    filepath:          fiftyone.core.fields.StringField
    tags:              fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:          fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    predictions:       fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    mistakenness:      fiftyone.core.fields.FloatField
    possible_missing:  fiftyone.core.fields.IntField
    possible_spurious: fiftyone.core.fields.IntField
View stages:
    1. SortBy(field_or_expr='mistakenness', reverse=True)


In [15]:
# Inspect some samples and detections
# This is the first detection of the first sample
print(mistake_view.first().ground_truth.detections[0])

<Detection: {
    'id': '60e4137cd10584dfdd7db82e',
    'attributes': BaseDict({}),
    'tags': BaseList([]),
    'label': 'Elephant',
    'bounding_box': BaseList([
        0.2568359375,
        0.3177083333333333,
        0.412109375,
        0.4739583333333333,
    ]),
    'mask': None,
    'confidence': None,
    'index': None,
    'supercategory': '',
    'iscrowd': 0,
    'mistakenness': 0.9535,
    'mistakenness_loc': 0.25206893883866816,
}>


Let's use the App to visually inspect the results:

In [16]:
# Show the samples we processed in rank order by the mistakenness
session.view = mistake_view

Another useful query is to find all objects that have a high mistakenness, lets say > 0.9:

In [17]:
session.view = dataset.filter_labels("ground_truth", F("mistakenness") > 0.9)

Looking through the results, we see some annotations that may be incorrect.

We can use a similar workflow to look at objects that may be localized poorly:

In [18]:
session.view = dataset.filter_labels("ground_truth", F("mistakenness_loc") > 0.9)

The `possible_missing` field can also be useful to sort by to find objects that the model detected that may have been missed by annotators.

Similarly, `possible_spurious` can be used to find instances of incorrect annotations.

In [19]:
session.view = dataset.match(F("possible_missing") > 0)

In [20]:
session.view = dataset.match(F("possible_spurious") > 0)

## Tagging and resolution

Any label or collection of labels can be tagged at any time in the sample grid or expanded sample view. In the expanded sample view, individual samples can be selected by clicking on them in the media player. We can, for example, tag a prediction as `missing` and any other predictions without an associated ground truth detection.

Labels with specific tags can then be selected with [select_labels()](https://voxel51.com/docs/fiftyone/api/fiftyone.core.collections.html?highlight=select_labels#fiftyone.core.collections.SampleCollection.select_labels) stage and sent off to assist in improving the annotations with your annotation provided of choice. FiftyOne currently offers integrations for both [Labelbox](https://voxel51.com/docs/fiftyone/api/fiftyone.utils.labelbox.html) and [Scale](https://voxel51.com/docs/fiftyone/api/fiftyone.utils.scale.html).

In [21]:
# A dataset can be filtered to only contain labels with certain tags
# Helpful for isolating labels with issues and sending off to an annotation provider
missing_ground_truth = dataset.select_labels(tags="missing")

In [22]:
session.view = missing_ground_truth

**REMEMBER**: Since you are using model predictions to guide the mistakenness process, the better your model, the more accurate the mistakenness suggestions. Additionally, using logits of confidence scores will also provide better results.